In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 10000)
pd.set_option("display.float_format", lambda x: f"{x:.10f}")


# Relative Strength Index (RSI)

In [15]:
def calculate_rsi(price_data, window=14, price_col='close'):
    """
    Correct RSI calculation.
    """
    delta = price_data[price_col].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


# Average True Range (ATR)

In [17]:
def calculate_atr(price_data, window=14):
    """
    Calculate ATR (Average True Range) using Wilder's RMA.

    Args:
        price_data (pd.DataFrame): DataFrame containing columns 'high', 'low', 'close'
        window (int): Lookback period (default: 14)

    Returns:
        pd.Series: ATR values
    """
    high = price_data['high']
    low = price_data['low']
    close = price_data['close']

    # Calculate True Range (TR)
    previous_close = close.shift(1)
    tr1 = high - low
    tr2 = abs(high - previous_close)
    tr3 = abs(low - previous_close)
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # ATR = RMA of True Range
    atr = true_range.ewm(alpha=1/window, adjust=False).mean()

    return atr

# Bollinger Bands

In [18]:
def calculate_bollinger_bands(price_data, window=20, std_multiplier=2, price_col='close'):
    """
    Calculate Bollinger Bands.

    Args:
        price_data (pd.DataFrame): DataFrame containing OHLC prices
        window (int): Lookback period for SMA (default: 20)
        std_multiplier (float): Standard deviation multiplier (default: 2)
        price_col (str): Column name for price (default: 'close')

    Returns:
        pd.DataFrame: DataFrame with 'bb_middle', 'bb_upper', 'bb_lower' columns
    """
    price = price_data[price_col]

    # Middle Band: SMA
    sma = price.rolling(window=window).mean()

    # Standard Deviation
    std = price.rolling(window=window).std()

    # Upper and Lower Bands
    upper_band = sma + std_multiplier * std
    lower_band = sma - std_multiplier * std

    # Return as DataFrame
    bb = pd.DataFrame({
        'bb_middle': sma,
        'bb_upper': upper_band,
        'bb_lower': lower_band
    })

    return bb


# Donchain Channels

In [19]:
def calculate_donchian_channel(price_data, window=20):
    """
    Calculate Donchian Channel.

    Args:
        price_data (pd.DataFrame): DataFrame with 'high' and 'low' columns
        window (int): Lookback period (default: 20)

    Returns:
        pd.DataFrame: DataFrame with 'dc_upper', 'dc_lower', 'dc_middle'
    """
    upper_band = price_data['high'].rolling(window=window).max()
    lower_band = price_data['low'].rolling(window=window).min()
    middle_band = (upper_band + lower_band) / 2

    dc = pd.DataFrame({
        'dc_upper': upper_band,
        'dc_lower': lower_band,
        'dc_middle': middle_band
    })

    return dc


# Trend

In [20]:
def determine_sma_trend(price_data, short_window=50, long_window=200, price_col='close'):
    """
    Determine trend based on SMA crossover.

    Args:
        price_data (pd.DataFrame): DataFrame containing price data
        short_window (int): Short SMA period (default: 13)
        long_window (int): Long SMA period (default: 50)
        price_col (str): Column name for price (default: 'close')

    Returns:
        pd.Series: Trend labels ('uptrend' or 'downtrend')
    """
    short_sma = price_data[price_col].rolling(window=short_window).mean()
    long_sma = price_data[price_col].rolling(window=long_window).mean()

    trend = np.where(short_sma > long_sma, 'uptrend', 'downtrend')

    return pd.Series(trend, index=price_data.index)


# Nadaraya Watson Envelope

In [21]:
def calculate_nadaraya_watson_envelope_optimized(df, bandwidth=8.0, multiplier=3.0, source_col='close', window_size=50):
    """
    Nadaraya-Watson Envelope'un repaint olmayan modunu hesaplar.
    Sadece hesaplanan 'nw_lower', 'nw', 'nw_upper' sütunlarını içeren bir DataFrame döndürür.

    Parametreler:
    df (pd.DataFrame): 'close', 'open', 'high', 'low', 'time' sütunlarına sahip DataFrame.
    bandwidth (float): Nadaraya-Watson için bant genişliği (h).
    multiplier (float): Zarfların genişliğini ayarlamak için çarpan.
    source_col (str): Hesaplama için kullanılacak kaynak sütun adı (örn: 'close').
    window_size (int): Nadaraya-Watson ve MAE hesaplamalarında kullanılacak maksimum geçmiş bar sayısı.

    Döndürür:
    pd.DataFrame: 'nw_lower', 'nw', 'nw_upper' sütunlarını içeren yeni bir DataFrame.
                  Bu DataFrame'in indeksleri orijinal df ile aynı olacaktır.
    """

    n_bars = len(df)
    source_data = df[source_col].values # NumPy array'ine dönüştürerek erişimi hızlandır

    # Gaussian kernel fonksiyonu
    def gauss(x, h):
        return np.exp(-(x**2) / (h * h * 2))

    # Ağırlıkları bir kere hesapla
    weights = np.array([gauss(i, bandwidth) for i in range(window_size)])
    weights_sum = np.sum(weights)

    nw_out_arr = np.full(n_bars, np.nan)
    nw_lower_arr = np.full(n_bars, np.nan)
    nw_upper_arr = np.full(n_bars, np.nan)

    for i in range(n_bars):
        if i < window_size - 1:
            continue


        weighted_sum = np.dot(source_data[i - window_size + 1 : i + 1], weights[::-1])

        current_nw_out = weighted_sum / weights_sum
        nw_out_arr[i] = current_nw_out

        if i >= window_size -1:
            abs_diffs = np.abs(source_data[i - window_size + 1 : i + 1] - nw_out_arr[i - window_size + 1 : i + 1])
            current_mae = np.mean(abs_diffs) * multiplier # ta.sma'ya benzer

            nw_lower_arr[i] = current_nw_out - current_mae
            nw_upper_arr[i] = current_nw_out + current_mae

    results_df = pd.DataFrame({
        'nw': nw_out_arr,
        'nw_upper': nw_upper_arr,
        'nw_lower': nw_lower_arr
    }, index=df.index)
    return results_df

# ADX

In [22]:
def add_adx(df, period=14):
    """
    DataFrame'e ADX (Average Directional Index) göstergesini ekler.

    Parametreler:
    df (pd.DataFrame): 'high', 'low' ve 'close' sütunlarını içeren DataFrame.
                       Sütun isimlerinin küçük harf olduğundan emin olun.
    period (int): ADX hesaplaması için kullanılacak periyot (varsayılan 14).

    Döndürür:
    pd.DataFrame: ADX, +DI, -DI ve DX sütunları eklenmiş DataFrame.
    """

    # Gerekli sütunların küçük harf kontrolü ve hata yönetimi
    required_cols = ['high', 'low', 'close']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"DataFrame'de '{', '.join(required_cols)}' sütunları bulunmalıdır ve küçük harf olmalıdır.")

    # Geçici çalışma DataFrame'i oluştur (orijinal df'i değiştirmemek için kopyala)
    df_adx = df.copy()

    # --- Yardımcı EMA Hesaplama Fonksiyonu ---
    def _calculate_ema(series, p):
        ema_values = [np.nan] * len(series)
        if len(series) < p:
            return pd.Series(ema_values, index=series.index)

        # İlk EMA değeri için SMA kullanılır
        ema_values[p - 1] = series.iloc[:p].mean()

        alpha = 2 / (p + 1)
        for i in range(p, len(series)):
            ema_values[i] = (series.iloc[i] * alpha) + (ema_values[i-1] * (1 - alpha))

        return pd.Series(ema_values, index=series.index)

    # --- 1. Adım: True Range (TR) Hesapla ---
    # TR = Max[(high - low), abs(high - prev_close), abs(low - prev_close)]
    df_adx['prev_close'] = df_adx['close'].shift(1)
    df_adx['high_low'] = df_adx['high'] - df_adx['low']
    df_adx['high_prev_close'] = abs(df_adx['high'] - df_adx['prev_close'])
    df_adx['low_prev_close'] = abs(df_adx['low'] - df_adx['prev_close'])
    df_adx['tr'] = df_adx[['high_low', 'high_prev_close', 'low_prev_close']].max(axis=1)

    # --- 2. Adım: Directional Movement (+DM ve -DM) Hesapla ---
    df_adx['prev_high'] = df_adx['high'].shift(1)
    df_adx['prev_low'] = df_adx['low'].shift(1)
    df_adx['up_move'] = df_adx['high'] - df_adx['prev_high']
    df_adx['down_move'] = df_adx['prev_low'] - df_adx['low']

    df_adx['+dm'] = np.where(
        (df_adx['up_move'] > df_adx['down_move']) & (df_adx['up_move'] > 0),
        df_adx['up_move'],
        0
    )
    df_adx['-dm'] = np.where(
        (df_adx['down_move'] > df_adx['up_move']) & (df_adx['down_move'] > 0),
        df_adx['down_move'],
        0
    )

    # --- 3. Adım: ATR, +DM ve -DM'nin Üssel Hareketli Ortalamalarını (EMA) Hesapla ---
    df_adx['tr_ema'] = _calculate_ema(df_adx['tr'], period)
    df_adx['+dm_ema'] = _calculate_ema(df_adx['+dm'], period)
    df_adx['-dm_ema'] = _calculate_ema(df_adx['-dm'], period)

    # --- 4. Adım: Yönsel İndeksler (DI) Hesapla ---
    df_adx['+di'] = (df_adx['+dm_ema'] / df_adx['tr_ema']) * 100
    df_adx['-di'] = (df_adx['-dm_ema'] / df_adx['tr_ema']) * 100

    # Sonsuzlukları ve sıfıra bölme hatalarını yönet
    df_adx['+di'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df_adx['-di'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df_adx.loc[df_adx['tr_ema'] == 0, ['+di', '-di']] = np.nan

    # --- 5. Adım: Yönsel Hareket İndeksi (DX) Hesapla ---
    df_adx['dx'] = (abs(df_adx['+di'] - df_adx['-di']) / (df_adx['+di'] + df_adx['-di'])) * 100
    df_adx['dx'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df_adx.loc[(df_adx['+di'] + df_adx['-di']) == 0, 'dx'] = np.nan

    # --- 6. Adım: ADX Hesapla (DX'in EMA'sı) ---
    df_adx['adx'] = _calculate_ema(df_adx['dx'], period)

    # İstenirse ara hesaplama sütunlarını temizle
    # df_adx.drop(columns=['prev_close', 'high_low', 'high_prev_close', 'low_prev_close',
    #                      'prev_high', 'prev_low', 'up_move', 'down_move',
    #                      'tr', '+dm', '-dm', 'tr_ema', '+dm_ema', '-dm_ema', 'dx'],
    #                     errors='ignore', inplace=True)

    return df_adx

# Candle

In [23]:
def candle(df):
    if df['close'] > df['open']:
        return'green'
    else:
        return 'red'

def classify_strength(row):
    if row['close'] > row['open']:
        if row['candle_strength'] > 1.1:
            return 'strong_bullish'
        elif row['candle_strength'] > 0.7:
            return 'medium_bullish'
        else:
            return 'weak_bullish'
    else:
        if row['candle_strength'] > 1.1:
            return 'strong_bearish'
        elif row['candle_strength'] > 0.7:
            return 'medium_bearish'
        else:
            return 'weak_bearish'




# DATAFRAME

In [24]:
df = pd.read_csv('/content/BINANCE_SUIUSDT, 15.csv')
df.columns = df.columns.str.lower()
df['time'] = pd.to_datetime(df['time'])

df.tail()

,time,open,high,low,close,volume
20376,2025-07-01 09:00:00+03:00,2.7872000000,2.7904000000,2.7782000000,2.7826000000,165642.8000000000
20377,2025-07-01 09:15:00+03:00,2.7825000000,2.7976000000,2.7816000000,2.7946000000,194876.8000000000
20378,2025-07-01 09:30:00+03:00,2.7946000000,2.7983000000,2.7902000000,2.7958000000,158671.6000000000
20379,2025-07-01 09:45:00+03:00,2.7959000000,2.8085000000,2.7875000000,2.8077000000,511417.4000000000
20380,2025-07-01 10:00:00+03:00,2.8077000000,2.8131000000,2.8059000000,2.8099000000,137065.7000000000


## Creating Main Columns

In [25]:
df['rsi'] = calculate_rsi(df)
df['atr'] = calculate_atr(df)

bb = calculate_bollinger_bands(df)
df['bb_middle'] = bb['bb_middle']
df['bb_upper'] = bb['bb_upper']
df['bb_lower'] = bb['bb_lower']

dc = calculate_donchian_channel(df, window=20)
df['dc_upper_20'] = dc['dc_upper']
df['dc_lower_20'] = dc['dc_lower']
df['dc_middle_20'] = dc['dc_middle']

dc = calculate_donchian_channel(df, window=50)
df['dc_upper_50'] = dc['dc_upper']
df['dc_lower_50'] = dc['dc_lower']
df['dc_middle_50'] = dc['dc_middle']


df['trend_50_200']=determine_sma_trend(df, short_window=50, long_window=200)

nw_bands = calculate_nadaraya_watson_envelope_optimized(df, bandwidth=8.0, multiplier=3.0, source_col='close', window_size=50)
df['nw'] = nw_bands['nw']
df['nw_upper'] = nw_bands['nw_upper']
df['nw_lower'] = nw_bands['nw_lower']

df['candle'] = df.apply(candle, axis=1)
df['candle_body'] = abs(df['close'] - df['open'])
df['candle_strength'] = df['candle_body'] / df['atr']
df['candle_class'] = df.apply(classify_strength, axis=1)

df = add_adx(df, period=14)

/tmp/ipython-input-22-3933902896.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_adx['+di'].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipython-input-22-3933902896.py:73: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

# Derivative Variables

## Donchian Channel Positions

In [26]:
df['dc_position_ratio_20'] = (df['close'] - df['dc_lower_20']) / (df['dc_upper_20'] - df['dc_lower_20']) * 100
df['dc_position_ratio_50'] = (df['close'] - df['dc_lower_50']) / (df['dc_upper_50'] - df['dc_lower_50']) * 100


df['dc_breakout_20'] = df['high'] > df['dc_upper_20']
df['dc_breakdown_20'] = df['low'] < df['dc_lower_20']

df['dc_breakout_50'] = df['high'] > df['dc_upper_50']
df['dc_breakdown_50'] = df['low'] < df['dc_lower_50']

## Percent ATR

In [30]:
df['pct_atr'] = df['atr'] / df['close'] * 100

# BB Strategies - 1 Or 3 Touch - Trend or Not

In [31]:
def bb_touch_signal(df, touch_count=1, trend_filter=False, trend_col='trend_50_200', trend_direction='uptrend'):
    """
    BB üst banda veya alt banda temas sayısına göre Long/Short sinyali.

    Params:
        df: DataFrame with BB bands & trend
        touch_count: int (1 veya 3)
        trend_filter: bool (True → trend şartı aranır)
        trend_col: str (ör. 'sma_trend')
        trend_direction: 'uptrend' veya 'downtrend'

    Returns:
        pd.Series: True where signal occurs
    """
    df['bb_touch_upper'] = df['high'] >= df['bb_upper']
    df['bb_touch_lower'] = df['low'] <= df['bb_lower']

    # Son 'touch_count' bardaki temas toplamı
    touch_upper = sum([df['bb_touch_upper'].shift(i+1) for i in range(touch_count)])
    touch_lower = sum([df['bb_touch_lower'].shift(i+1) for i in range(touch_count)])

    signal_long = touch_upper >= touch_count
    signal_short = touch_lower >= touch_count

    if trend_filter:
        signal_long = signal_long & (df[trend_col] == trend_direction)
        signal_short = signal_short & (df[trend_col] != trend_direction)

    return pd.Series(signal_long, index=df.index), pd.Series(signal_short, index=df.index)


# DC Strategies - Trend or Not

In [32]:
def dc_breakout_signal(df, dc_upper='dc_upper_20', dc_lower='dc_lower_20',
                       trend_filter=False, trend_col='trend_50_200', trend_direction='uptrend'):
    """
    DC breakout sinyali.

    Params:
        dc_upper: Üst kanal kolonu
        dc_lower: Alt kanal kolonu
        trend_filter: Trend filtresi kullanılacak mı?
        trend_col: Trend kolonu ismi
        trend_direction: Beklenen trend yönü ('uptrend' veya 'downtrend')

    Returns:
        long_signal, short_signal: pd.Series (bool)
    """
    # DC breakout: şu anki high, bir önceki üst bandın üstüne çıktı mı?
    breakout_upper = df['high'] > df[dc_upper].shift(1)
    breakout_lower = df['low'] < df[dc_lower].shift(1)

    signal_long = breakout_upper
    signal_short = breakout_lower

    if trend_filter:
        signal_long &= (df[trend_col] == trend_direction)
        signal_short &= (df[trend_col] != trend_direction)

    return pd.Series(signal_long, index=df.index), pd.Series(signal_short, index=df.index)



In [33]:
# DC 50 breakout
long_dc50, short_dc50 = dc_breakout_signal(df, 'dc_upper_50', 'dc_lower_50', trend_filter=True)
df['dc_breakout_50'] = long_dc50
df['dc_breakdown_50'] = short_dc50


# BB üç kere temas long/short
long3, short3 = bb_touch_signal(df, touch_count=3, trend_filter=True)
df['bb_3_touch_long'] = long3
df['bb_3_touch_short'] = short3



# BB.ELF Clean Signal

In [35]:
# Long tarafı için
no_long_touch_last_10 = df['bb_3_touch_long'].shift(1).rolling(window=10).sum() == 0
df['bb_3_touch_long_clean'] = df['bb_3_touch_long'] & no_long_touch_last_10

# Short tarafı için
no_short_touch_last_10 = df['bb_3_touch_short'].shift(1).rolling(window=10).sum() == 0
df['bb_3_touch_short_clean'] = df['bb_3_touch_short'] & no_short_touch_last_10


# DC Clean Signal

In [36]:
df['dc_breakout_clean_50'] = df['dc_breakout_50'] & (
    df['dc_breakout_50'].shift(1).rolling(window=10).sum() == 0)

df['dc_breakdown_clean_50'] = df['dc_breakdown_50'] & (
    df['dc_breakdown_50'].shift(1).rolling(window=10).sum() == 0)

## BB3 Strategy

In [37]:
# BB3 ELF 15 İÇİN
# Başlangıç değerleri
start_capital = 100000
capital = start_capital
x_unit = 1000  # 1x başına 1000 dolar

# Kayıtlar
equity_data = []
trades = []

# Pozisyon takibi
current_position = None  # (entry_idx, direction, entry_price)

# Gerekli percentiller
low_atr = df['pct_atr'].quantile(0.2)
high_atr = df['pct_atr'].quantile(0.8)

for idx, row in df.iterrows():
    # Sinyaller
    signal_long = (row['bb_3_touch_long_clean'] and (row['dc_position_ratio_20'] > 60) and  (row['rsi'] > 50) and (row['close'] > row['nw']) and (row['close'] < row['nw_upper']) and
                   (row['close'] > row['bb_middle']) and (row['adx'] > 25) and (row['adx'] < 60) and row['candle_class'] in ['weak_bearish', 'weak_bullish', 'medium_bullish', 'strong_bullish']
                   and low_atr < row['pct_atr'] < high_atr)

    signal_short = (row['bb_3_touch_short_clean'] and (row['dc_position_ratio_20'] < 40) and (row['rsi'] < 50) and (row['close'] < row['bb_middle']) and (row['close'] < row['nw']) and
                    (row['close'] > row['nw_lower']) and  (row['adx'] > 25) and (row['adx'] < 60) and row['candle_class'] in ['weak_bearish', 'weak_bullish', 'medium_bearish', 'strong_bearish']
                    and low_atr < row['pct_atr'] < high_atr)

    # Mevcut pozisyon varsa TP/SL kontrolü
    if current_position:
        entry_idx, direction, entry_price = current_position
        entry_row = df.loc[entry_idx]

        if direction == 'long':
            tp_bars = entry_row['4x_bar']
            sl_bars = entry_row['-1x_bar']
            tp_hit = pd.notna(tp_bars) and (idx - entry_idx == tp_bars)
            sl_hit = pd.notna(sl_bars) and (idx - entry_idx == sl_bars)
            pnl_x = 4 if tp_hit else -1 if sl_hit else 0

        else:  # short
            tp_bars = entry_row['-4x_bar']
            sl_bars = entry_row['2x_bar']
            tp_hit = pd.notna(tp_bars) and (idx - entry_idx == tp_bars)
            sl_hit = pd.notna(sl_bars) and (idx - entry_idx == sl_bars)
            pnl_x = 2 if tp_hit else -1 if sl_hit else 0

        if tp_hit or sl_hit:
            exit_reason = 'TP' if tp_hit else 'SL'
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], df.loc[entry_idx, 'pct_atr']
            ))
            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

    # Yeni sinyal gelirse önce varsa pozisyonu kapat
    if signal_long:
        if current_position:
            exit_price = row['close']
            pnl = exit_price - entry_price  # long pozisyon
            pnl_pct = pnl / entry_price * 100
            pnl_x = pnl_pct / (row['x'] / row['close'] * 100)
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            exit_reason = 'Signal'
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], df.loc[entry_idx, 'pct_atr']
            ))
            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

        # Yeni long aç
        direction = 'long'
        current_position = (idx, direction, row['close'])

    elif signal_short:
        if current_position:
            exit_price = row['close']
            pnl = entry_price - exit_price  # short pozisyon
            pnl_pct = pnl / entry_price * 100
            pnl_x = pnl_pct / (row['x'] / row['close'] * 100) / 2  # shortlarda 2'ye böl
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            exit_reason = 'Signal'
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], df.loc[entry_idx, 'pct_atr']
            ))

            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

        # Yeni short aç
        direction = 'short'
        current_position = (idx, direction, row['close'])

# DataFrame oluşturma
trades_df = pd.DataFrame(trades, columns=[
    'Pnl_Dollar', 'Pnl_X', 'Direction', 'Entry_Idx', 'Exit_Idx', 'Exit_Reason', 'Entry_Time', 'Exit_Time', 'Entry_pct_atr'
])
trades_df['Result'] = trades_df['Pnl_Dollar'].apply(lambda x: 'Win' if x > 0 else 'Loss')

equity_df = pd.DataFrame(equity_data)
equity_df.to_csv("equity_curve.csv", index=False)
trades_df.to_csv("trades_results.csv", index=False)

# Sonuçlar
print("Başlangıç Sermaye:", start_capital)
print("Bitiş Sermaye:", capital)
total_pnl = capital - start_capital
print(f"Toplam Kazanç: {total_pnl:.2f} $ → %{(total_pnl / start_capital * 100):.2f}")
print(f"Toplam Kazanç: {trades_df['Pnl_X'].sum():.2f} X")
print(f"Toplam Kazanç 100 İşlemde: %{round((trades_df['Pnl_X'].sum()) / (len(equity_df)) * 100, 2)} X")
print(f"İşlem Sayısı: {len(equity_df)}, Komisyon Miktarı: {round(len(equity_df) * 0.04)}-{round(len(equity_df) * 0.05)} X")

# Özet tablo
print("\nİşlem Sonuçları Özeti:")
summary = trades_df.groupby(['Exit_Reason', 'Result'])['Pnl_Dollar'].agg(['count', 'sum'])
print(summary)

print("\nSignal Ortalama X Kazancı:", trades_df[trades_df['Exit_Reason'] == 'Signal']['Pnl_X'].mean())

# Long-short dağılımı
print("\nLong/Short Sayısı:")
print(trades_df['Direction'].value_counts())

# Long işlemler için TP ve SL
long_df = trades_df[trades_df['Direction'] == 'long']
long_tp = len(long_df[long_df['Exit_Reason'] == 'TP'])
long_sl = len(long_df[long_df['Exit_Reason'] == 'SL'])
long_total = long_tp + long_sl
long_win_rate = long_tp / long_total * 100 if long_total else 0

# Short işlemler için TP ve SL
short_df = trades_df[trades_df['Direction'] == 'short']
short_tp = len(short_df[short_df['Exit_Reason'] == 'TP'])
short_sl = len(short_df[short_df['Exit_Reason'] == 'SL'])
short_total = short_tp + short_sl
short_win_rate = short_tp / short_total * 100 if short_total else 0

print("\nLong İşlemler:\nTP:", long_tp, "SL:", long_sl, f"Win Rate: %{long_win_rate:.2f}")
print("Short İşlemler:\nTP:", short_tp, "SL:", short_sl, f"Win Rate: %{short_win_rate:.2f}")

print("\n--- Percent ATR Analizi ---")

# Long işlemler
long_wins = long_df[long_df['Result'] == 'Win']
long_losses = long_df[long_df['Result'] == 'Loss']

print(f"Long - Win ortalama pct_atr: {long_wins['Entry_pct_atr'].mean():.4f}")
print(f"Long - Loss ortalama pct_atr: {long_losses['Entry_pct_atr'].mean():.4f}")

# Short işlemler
short_wins = short_df[short_df['Result'] == 'Win']
short_losses = short_df[short_df['Result'] == 'Loss']

print(f"Short - Win ortalama pct_atr: {short_wins['Entry_pct_atr'].mean():.4f}")
print(f"Short - Loss ortalama pct_atr: {short_losses['Entry_pct_atr'].mean():.4f}")


# Equity grafiği
plt.figure(figsize=(12, 6))
plt.plot(pd.to_datetime(equity_df['Time']), equity_df['Capital'])
plt.title("Equity Curve Over Time")
plt.xlabel("Tarih")
plt.ylabel("Sermaye ($)")
plt.grid(True)
plt.tight_layout()
plt.show()


KeyError: '-4x_bar'

## DONCHAIN CHANNELS 50 Strategy

In [ ]:
# DONCHAIN CHANNELS 50 15DK İÇİN
# Başlangıç değerleri
start_capital = 100000
capital = start_capital
x_unit = 1000  # 1x başına 1000 dolar

# Kayıtlar
equity_data = []
trades = []

# Pozisyon takibi
current_position = None  # (entry_idx, direction, entry_price)

# Gerekli percentiller
low_atr = df['pct_atr'].quantile(0.2)
high_atr = df['pct_atr'].quantile(0.8)

for idx, row in df.iterrows():
    # Sinyaller
    signal_long = (row['dc_breakout_clean_50'] and (row['dc_position_ratio_20'] > 60) and  (row['rsi'] > 50) and (row['close'] > row['nw']) and (row['close'] < row['nw_upper']) and
                   (row['close'] > row['bb_middle']) and (row['adx'] > 25) and (row['adx'] < 60) and row['candle_class'] in ['weak_bearish', 'weak_bullish', 'medium_bullish', 'strong_bullish']
                   and low_atr < row['pct_atr'] < high_atr)

    signal_short = (row['dc_breakdown_clean_50'] and (row['dc_position_ratio_20'] < 40) and (row['rsi'] < 50) and (row['close'] < row['bb_middle']) and (row['close'] < row['nw']) and
                    (row['close'] > row['nw_lower']) and  (row['adx'] > 25) and (row['adx'] < 60) and row['candle_class'] in ['weak_bearish', 'weak_bearish', 'medium_bearish', 'strong_bearish']
                    and low_atr < row['pct_atr'] < high_atr)


    # Mevcut pozisyon varsa TP/SL kontrolü
    if current_position:
        entry_idx, direction, entry_price = current_position
        entry_row = df.loc[entry_idx]

        if direction == 'long':
            tp_bars = entry_row['4x_bar']
            sl_bars = entry_row['-1x_bar']
            tp_hit = pd.notna(tp_bars) and (idx - entry_idx == tp_bars)
            sl_hit = pd.notna(sl_bars) and (idx - entry_idx == sl_bars)
            pnl_x = 4 if tp_hit else -1 if sl_hit else 0

        else:  # short
            tp_bars = entry_row['-4x_bar']
            sl_bars = entry_row['2x_bar']
            tp_hit = pd.notna(tp_bars) and (idx - entry_idx == tp_bars)
            sl_hit = pd.notna(sl_bars) and (idx - entry_idx == sl_bars)
            pnl_x = 2 if tp_hit else -1 if sl_hit else 0

        if tp_hit or sl_hit:
            exit_reason = 'TP' if tp_hit else 'SL'
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], dff.loc[entry_idx, 'pct_atr']
            ))
            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

    # Yeni sinyal gelirse önce varsa pozisyonu kapat
    if signal_long:
        if current_position:
            exit_price = row['close']
            pnl = exit_price - entry_price  # long pozisyon
            pnl_pct = pnl / entry_price * 100
            pnl_x = pnl_pct / (row['x'] / row['close'] * 100)
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            exit_reason = 'Signal'
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], df.loc[entry_idx, 'pct_atr']
            ))
            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

        # Yeni long aç
        direction = 'long'
        current_position = (idx, direction, row['close'])

    elif signal_short:
        if current_position:
            exit_price = row['close']
            pnl = entry_price - exit_price  # short pozisyon
            pnl_pct = pnl / entry_price * 100
            pnl_x = pnl_pct / (row['x'] / row['close'] * 100) / 2  # shortlarda 2'ye böl
            pnl_dollar = pnl_x * x_unit

            capital += pnl_dollar
            exit_reason = 'Signal'
            trades.append((
                pnl_dollar, pnl_x, direction, entry_idx, idx, exit_reason,
                df.loc[entry_idx, 'time'], row['time'], df.loc[entry_idx, 'pct_atr']
            ))

            equity_data.append({'Capital': capital, 'Time': row['time']})
            current_position = None

        # Yeni short aç
        direction = 'short'
        current_position = (idx, direction, row['close'])

# DataFrame oluşturma
trades_df = pd.DataFrame(trades, columns=[
    'Pnl_Dollar', 'Pnl_X', 'Direction', 'Entry_Idx', 'Exit_Idx', 'Exit_Reason', 'Entry_Time', 'Exit_Time', 'Entry_pct_atr'
])
trades_df['Result'] = trades_df['Pnl_Dollar'].apply(lambda x: 'Win' if x > 0 else 'Loss')

equity_df = pd.DataFrame(equity_data)
equity_df.to_csv("equity_curve.csv", index=False)
trades_df.to_csv("trades_results.csv", index=False)

# Sonuçlar
print("Başlangıç Sermaye:", start_capital)
print("Bitiş Sermaye:", capital)
total_pnl = capital - start_capital
print(f"Toplam Kazanç: {total_pnl:.2f} $ → %{(total_pnl / start_capital * 100):.2f}")
print(f"Toplam Kazanç: {trades_df['Pnl_X'].sum():.2f} X")
print(f"Toplam Kazanç 100 İşlemde: %{round((trades_df['Pnl_X'].sum()) / len(equity_df) * 100, 2)} X")
print(f"İşlem Sayısı: {len(equity_df)}, Komisyon Miktarı: {round(len(equity_df) * 0.04)}-{round(len(equity_df) * 0.05)} X")

# Özet tablo
print("\nİşlem Sonuçları Özeti:")
summary = trades_df.groupby(['Exit_Reason', 'Result'])['Pnl_Dollar'].agg(['count', 'sum'])
print(summary)

print("\nSignal Ortalama X Kazancı:", trades_df[trades_df['Exit_Reason'] == 'Signal']['Pnl_X'].mean())

# Long-short dağılımı
print("\nLong/Short Sayısı:")
print(trades_df['Direction'].value_counts())

# Long işlemler için TP ve SL
long_df = trades_df[trades_df['Direction'] == 'long']
long_tp = len(long_df[long_df['Exit_Reason'] == 'TP'])
long_sl = len(long_df[long_df['Exit_Reason'] == 'SL'])
long_total = long_tp + long_sl
long_win_rate = long_tp / long_total * 100 if long_total else 0

# Short işlemler için TP ve SL
short_df = trades_df[trades_df['Direction'] == 'short']
short_tp = len(short_df[short_df['Exit_Reason'] == 'TP'])
short_sl = len(short_df[short_df['Exit_Reason'] == 'SL'])
short_total = short_tp + short_sl
short_win_rate = short_tp / short_total * 100 if short_total else 0

print("\nLong İşlemler:\nTP:", long_tp, "SL:", long_sl, f"Win Rate: %{long_win_rate:.2f}")
print("Short İşlemler:\nTP:", short_tp, "SL:", short_sl, f"Win Rate: %{short_win_rate:.2f}")

print("\n--- Percent ATR Analizi ---")

# Long işlemler
long_wins = long_df[long_df['Result'] == 'Win']
long_losses = long_df[long_df['Result'] == 'Loss']

print(f"Long - Win ortalama pct_atr: {long_wins['Entry_pct_atr'].mean():.4f}")
print(f"Long - Loss ortalama pct_atr: {long_losses['Entry_pct_atr'].mean():.4f}")

# Short işlemler
short_wins = short_df[short_df['Result'] == 'Win']
short_losses = short_df[short_df['Result'] == 'Loss']

print(f"Short - Win ortalama pct_atr: {short_wins['Entry_pct_atr'].mean():.4f}")
print(f"Short - Loss ortalama pct_atr: {short_losses['Entry_pct_atr'].mean():.4f}")


# Equity grafiği
plt.figure(figsize=(12, 6))
plt.plot(pd.to_datetime(equity_df['Time']), equity_df['Capital'])
plt.title("Equity Curve Over Time")
plt.xlabel("Tarih")
plt.ylabel("Sermaye ($)")
plt.grid(True)
plt.tight_layout()
plt.show()
